In [2]:
# imports
import pandas as pd
import numpy as np

# load data
data = pd.read_excel("data/tagesschau_articles.xlsx")

In [4]:
# check if date of URL matches time text

# format time string to datetime obj and drop string from df
data["date_and_time"] = pd.to_datetime(data["time_text"], format="%d.%m.%Y %H:%M Uhr")
data.drop(columns=["time_text"], inplace=True)

# format time strings to datetime
data["date"] = pd.to_datetime(data["date"], format="%d/%m/%Y")


# store values where URL date is different to date text
data["proofdate"] = data["date_and_time"].dt.date
not_matching = data["date"].compare(data["proofdate"])
data.iloc[not_matching.index]

,date,article,title,date_and_time,proofdate
79,2013-06-01,"Bayern München holt Tripel, Hochwassersituatio...",tagesthemen,2013-06-02 00:03:00,2013-06-02
778,2014-05-17,"Überflutungen in Osteuropa, Zweiter Runder Tis...",tagesthemen,2014-05-18 00:12:00,2014-05-18
2252,2016-05-21,Türkei lässt hochqualifizierte Flüchtlinge nic...,tagesthemen,2016-05-22 00:30:00,2016-05-22
2296,2016-06-12,"Schießerei in Nachtklub in Orlando, Vor Ort in...",tagesthemen,2016-06-13 00:27:00,2016-06-13
2813,2017-02-24,US-Präsident Trump bei der Jahresversammlung d...,tagesthemen,2017-02-25 00:10:00,2017-02-25
3358,2017-11-19,FDP bricht Jamaika-Verhandlungen ab,tagesthemen extra,2017-11-20 00:00:00,2017-11-20


Seems like all these episodes are just misclassified as the day before, so lets just use the date from the timestring since its more accurate

In [5]:
data["date"] = data["proofdate"]
data.drop(columns=["proofdate"], inplace=True)

In [6]:
# adding year, month, quarter, day, weekday, and timeslot as military time
data["year"] = data["date_and_time"].dt.year
data["month"] = data["date_and_time"].dt.month
data["quarter"] = data["date_and_time"].dt.quarter.astype(str) + "/" + data["year"].astype(str)
data["day"] = data["date_and_time"].dt.day
data["weekday"] = data["date_and_time"].dt.dayofweek
data["timeslot"] = data["date_and_time"].dt.strftime("%H%M").astype(int) - data["date_and_time"].dt.strftime("%H%M").astype(int) % 20

In [7]:
# lets look for duplicates in the data
data.groupby("date_and_time").size().nlargest(5)

date_and_time
2015-01-19 22:15:00    2
2013-04-22 20:00:00    1
2013-04-22 22:15:00    1
2013-04-23 20:00:00    1
2013-04-23 22:15:00    1
dtype: int64

Seems like there is a duplicate episode lets make sure it really is the same and if so, remove it

In [8]:
data[data["date_and_time"] == "2015-01-19 22:15:00"]

,date,article,title,date_and_time,year,month,quarter,day,weekday,timeslot
1271,2015-01-19,"Diskussion um Versammlungsfreiheit, Der Kommen...",tagesthemen,2015-01-19 22:15:00,2015,1,1/2015,19,0,2200
1272,2015-01-19,"Diskussion um Versammlungsfreiheit, Der Kommen...",tagesthemen,2015-01-19 22:15:00,2015,1,1/2015,19,0,2200


In [9]:
# remove duplicate episode
data.drop(index=1271, inplace=True)

In [10]:
# add len of description to data
data["desc_length"] = data["article"].str.len()

# add number of episode of day to dataframe
episodes_that_day = data.groupby("date").size().to_frame()
episodes_that_day.columns = ["episodes_that_day"]
data = data.merge(episodes_that_day, how="left", on="date")

In [11]:
# format article to different topics and add number of topics that episode
data["article"] = data["article"].str.split(",")
data["num_topics"] = data["article"].str.len()

In [12]:
# unstack "article" column to every topic
stacked_column = "article"
other_columns = data.columns.difference([stacked_column])
all_topics = [topic for sublist in data[stacked_column] for topic in sublist]

unstacked_df = {
    col: np.repeat(data[col], data["num_topics"]) for col in other_columns
}
unstacked_df["topic"] = all_topics
data = pd.DataFrame(unstacked_df)

# format title column so they are rightly classified
data["title"] = data["title"].str.lower().str.replace(" ", "")

data["topic"] = data["topic"].astype("str")

In [13]:
# save cleaned data to csv
data.to_excel("data/cleaned_data.xlsx", index=False)